In [33]:
from random import randint, randrange


class Node:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.children = []


class BTree:
    def __init__(self, t):
        """
        Initializing the B-Tree
        :param t: Order.
        """
        self.root = Node(True)
        self.t = t

    def printTree(self, x, lists, lvl=0):
        """
        Prints the complete B-Tree
        :param x: Root node.
        :param lvl: Current level.
        """
        #print(lvl,end=" ")
        lvl += 1
        if len(x.children) > 0:
            for i in x.children:
                self.printTree(i, lists, lvl)
        lists.append(lvl)

    def search(self, k, x=None):
        """
        Search for key 'k' at position 'x'
        :param k: The key to search for.
        :param x: The position to search from. If not specified, then search occurs from the root.
        :return: 'None' if 'k' is not found. Otherwise returns a tuple of (node, index) at which the key was found.
        """
        if x is not None:
            i = 0
            while i < len(x.keys) and k > x.keys[i][0]:
                i += 1
            if i < len(x.keys) and k == x.keys[i][0]:
                return x, i
            elif x.leaf:
                return None
            else:
                # Search its children
                return self.search(k, x.children[i])
        else:
            # Search the entire tree
            return self.search(k, self.root)

    def insert(self, k):
        """
        Calls the respective helper functions for insertion into B-Tree
        :param k: The key to be inserted.
        """
        root = self.root
        # If a node is full, split the child
        if len(root.keys) == (2 * self.t) - 1:
            temp = Node()
            self.root = temp
            # Former root becomes 0'th child of new root 'temp'
            temp.children.insert(0, root)
            self._splitChild(temp, 0)
            self._insertNonFull(temp, k)
        else:
            self._insertNonFull(root, k)

    def _insertNonFull(self, x, k):
        """
        Inserts a key in a non-full node
        :param x: The key to be inserted.
        :param k: The position of node.
        """
        i = len(x.keys) - 1
        if x.leaf:
            x.keys.append((None, None))
            while i >= 0 and k[0] < x.keys[i][0]:
                x.keys[i + 1] = x.keys[i]
                i -= 1
            x.keys[i + 1] = k
        else:
            while i >= 0 and k[0] < x.keys[i][0]:
                i -= 1
            i += 1
            if len(x.children[i].keys) == (2 * self.t) - 1:
                self._splitChild(x, i)
                if k[0] > x.keys[i][0]:
                    i += 1
            self._insertNonFull(x.children[i], k)

    def _splitChild(self, x, i):
        """
        Splits the child of node at 'x' from index 'i'
        :param x: Parent node of the node to be split.
        :param i: Index value of the child.
        """
        t = self.t
        y = x.children[i]
        z = Node(y.leaf)
        x.children.insert(i + 1, z)
        x.keys.insert(i, y.keys[t - 1])
        z.keys = y.keys[t: (2 * t) - 1]
        y.keys = y.keys[0: t - 1]
        if not y.leaf:
            z.children = y.children[t: 2 * t]
            y.children = y.children[0: t - 1]

    def delete(self, x, k):
        """
        Calls the respective helper functions for deletion from B-Tree
        :param x: The node, according to whose relative position, helper functions are called.
        :param k: The key to be deleted.
        """
        t = self.t
        i = 0
        while i < len(x.keys) and k[0] > x.keys[i][0]:
            i += 1
        # Deleting the key if the node is a leaf
        if x.leaf:
            if i < len(x.keys) and x.keys[i][0] == k[0]:
                x.keys.pop(i)
                return
            return

        # Calling '_deleteInternalNode' when x is an internal node and contains the key 'k'
        if i < len(x.keys) and x.keys[i][0] == k[0]:
            return self._deleteInternalNode(x, k, i)
        # Recursively calling 'delete' on x's children
        elif len(x.children[i].keys) >= t:
            self.delete(x.children[i], k)
        # Ensuring that a child always has atleast 't' keys
        else:
            if i != 0 and i + 2 < len(x.children):
                if len(x.children[i - 1].keys) >= t:
                    self._deleteSibling(x, i, i - 1)
                elif len(x.children[i + 1].keys) >= t:
                    self._deleteSibling(x, i, i + 1)
                else:
                    self._deleteMerge(x, i, i + 1)
            elif i == 0:
                if len(x.children[i + 1].keys) >= t:
                    self._deleteSibling(x, i, i + 1)
                else:
                    self._deleteMerge(x, i, i + 1)
            elif i + 1 == len(x.children):
                if len(x.children[i - 1].keys) >= t:
                    self._deleteSibling(x, i, i - 1)
                else:
                    self._deleteMerge(x, i, i - 1)
            self.delete(x.children[i], k)

    def _deleteInternalNode(self, x, k, i):
        """
        Deletes internal node
        :param x: The internal node in which key 'k' is present.
        :param k: The key to be deleted.
        :param i: The index position of key in the list
        """
        t = self.t
        # Deleting the key if the node is a leaf
        if x.leaf:
            if x.keys[i][0] == k[0]:
                x.keys.pop(i)
                return
            return

        # Replacing the key with its predecessor and deleting predecessor
        if len(x.children[i].keys) >= t:
            x.keys[i] = self._deletePredecessor(x.children[i])
            return
        # Replacing the key with its successor and deleting successor
        elif len(x.children[i + 1].keys) >= t:
            x.keys[i] = self._deleteSuccessor(x.children[i + 1])
            return
        # Merging the child, its left sibling and the key 'k'
        else:
            self._deleteMerge(x, i, i + 1)
            self._deleteInternalNode(x.children[i], k, self.t - 1)

    def _deletePredecessor(self, x):
        """
        Deletes predecessor of key 'k' which is to be deleted
        :param x: Node
        :return: Predecessor of key 'k' which is to be deleted
        """
        if x.leaf:
            return x.keys.pop()
        n = len(x.keys) - 1
        if len(x.children[n].keys) >= self.t:
            self._deleteSibling(x, n + 1, n)
        else:
            self._deleteMerge(x, n, n + 1)
        self._deletePredecessor(x.children[n])

    def _deleteSuccessor(self, x):
        """
        Deletes successor of key 'k' which is to be deleted
        :param x: Node
        :return: Successor of key 'k' which is to be deleted
        """
        if x.leaf:
            return x.keys.pop(0)
        if len(x.children[1].keys) >= self.t:
            self._deleteSibling(x, 0, 1)
        else:
            self._deleteMerge(x, 0, 1)
        self._deleteSuccessor(x.children[0])

    def _deleteMerge(self, x, i, j):
        """
        Merges the children of x and one of its own keys
        :param x: Parent node
        :param i: The index of one of the children
        :param j: The index of one of the children
        """
        cNode = x.children[i]

        # Merging the x.children[i], x.children[j] and x.keys[i]
        if j > i:
            rsNode = x.children[j]
            cNode.keys.append(x.keys[i])
            # Assigning keys of right sibling node to child node
            for k in range(len(rsNode.keys)):
                cNode.keys.append(rsNode.keys[k])
                if len(rsNode.children) > 0:
                    cNode.children.append(rsNode.children[k])
            if len(rsNode.children) > 0:
                cNode.children.append(rsNode.children.pop())
            new = cNode
            x.keys.pop(i)
            x.children.pop(j)
        # Merging the x.children[i], x.children[j] and x.keys[i]
        else:
            lsNode = x.children[j]
            lsNode.keys.append(x.keys[j])
            # Assigning keys of left sibling node to child node
            for i in range(len(cNode.keys)):
                lsNode.keys.append(cNode.keys[i])
                if len(lsNode.children) > 0:
                    lsNode.children.append(cNode.children[i])
            if len(lsNode.children) > 0:
                lsNode.children.append(cNode.children.pop())
            new = lsNode
            x.keys.pop(j)
            x.children.pop(i)

        # If x is root and is empty, then re-assign root
        if x == self.root and len(x.keys) == 0:
            self.root = new

    @staticmethod
    def _deleteSibling(x, i, j):
        """
        Borrows a key from j'th child of x and appends it to i'th child of x
        :param x: Parent node
        :param i: The index of one of the children
        :param j: The index of one of the children
        """
        cNode = x.children[i]
        if i < j:
            # Borrowing key from right sibling of the child
            rsNode = x.children[j]
            cNode.keys.append(x.keys[i])
            x.keys[i] = rsNode.keys[0]
            if len(rsNode.children) > 0:
                cNode.children.append(rsNode.children[0])
                rsNode.children.pop(0)
            rsNode.keys.pop(0)
        else:
            # Borrowing key from left sibling of the child
            lsNode = x.children[j]
            cNode.keys.insert(0, x.keys[i - 1])
            x.keys[i - 1] = lsNode.keys.pop()
            if len(lsNode.children) > 0:
                cNode.children.insert(0, lsNode.children.pop())



In [34]:
import time
import numpy as np

NUM = 1000

for i in range(10):

    t = BTree(5)
    lists = []
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    
    for num in range(NUM):
        t.insert((num, nums[num]))
    
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    t.printTree(t.root,lists)
    print(i+1,"번째 트리 높이: ", max(lists))
    print()
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(t.root,(randNum,))
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  0.005018472671508789
1 번째 트리 높이:  4

1 - 1 번째 탐색, 찾는 숫자 4738035 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 9400574 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 3317205 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 9038851 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 4440858 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.003988981246948242
2 번째 트리 높이:  4

2 - 1 번째 탐색, 찾는 숫자 8596632 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 469944 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 7941236 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 9151922 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 8746609 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.003988742828369141
3 번째 트리 높이:  4

3 - 1 번째 탐색, 찾는 숫자 2079079 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 8446999 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 5209107 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 7106152 탐색+삭제 시간: 0.0009992122650146484
3 - 5 번째 탐색, 찾는 숫자 5335829 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.003972530364990234
4 번째 트리 높이:  4

4 - 1 번째 탐색, 찾는 숫자 4591327 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 6679927 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 3243121 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 7136981 탐색+

In [55]:
import time
import numpy as np

NUM = 10000

for i in range(10):

    t = BTree(5)
    lists = []
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    
    for num in range(NUM):
        t.insert((num, nums[num]))
    
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    t.printTree(t.root,lists)
    print(i+1,"번째 트리 높이: ", max(lists))
    print()
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        
        try:
            t.delete(t.root,(1*j,))
            print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)

        except IndexError:
            print("")
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  0.05585074424743652
1 번째 트리 높이:  6

1 - 1 번째 탐색, 찾는 숫자 6195629 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 906172 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 289017 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 5410363 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 6896691 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.03233003616333008
2 번째 트리 높이:  6

2 - 1 번째 탐색, 찾는 숫자 8214354 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 2989386 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 8021866 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 9396643 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 2050708 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.06248760223388672
3 번째 트리 높이:  6

3 - 1 번째 탐색, 찾는 숫자 1882062 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 3840427 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 8605443 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 5423931 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 2068297 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.03124237060546875
4 번째 트리 높이:  6

4 - 1 번째 탐색, 찾는 숫자 1975289 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 1585116 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 5780801 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 2685273 탐색+삭제 시간: 0.0
4 - 5 번째 탐색,

In [38]:
import time
import numpy as np

NUM = 100000

for i in range(10):

    t = BTree(5)
    lists = []
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    
    for num in range(NUM):
        t.insert((num, nums[num]))
    
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    t.printTree(t.root,lists)
    print(i+1,"번째 트리 높이: ", max(lists))
    print()
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(t.root,(randNum,))
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  0.4789879322052002
1 번째 트리 높이:  7

1 - 1 번째 탐색, 찾는 숫자 4231985 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 8155430 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 8505516 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 5620708 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 8581598 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.6088547706604004
2 번째 트리 높이:  7

2 - 1 번째 탐색, 찾는 숫자 9918386 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 549577 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 3130591 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 9275132 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 67700 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.6054074764251709
3 번째 트리 높이:  7

3 - 1 번째 탐색, 찾는 숫자 6160003 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 4003482 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 4970722 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 1070963 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 8631597 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.7177326679229736
4 번째 트리 높이:  7

4 - 1 번째 탐색, 찾는 숫자 8753979 탐색+삭제 시간: 0.000997304916381836
4 - 2 번째 탐색, 찾는 숫자 1953312 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 4813764 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 8016772 탐색+삭제 시간: 0.0


In [39]:
import time
import numpy as np

NUM = 1000000

for i in range(10):

    t = BTree(5)
    lists = []
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    
    for num in range(NUM):
        t.insert((num, nums[num]))
    
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    t.printTree(t.root,lists)
    print(i+1,"번째 트리 높이: ", max(lists))
    print()
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(t.root,(randNum,))
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  6.51168155670166
1 번째 트리 높이:  9

1 - 1 번째 탐색, 찾는 숫자 1696495 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 9737036 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 2775367 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 8800419 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 7281399 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  6.087364912033081
2 번째 트리 높이:  9

2 - 1 번째 탐색, 찾는 숫자 566318 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 8020562 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 5274915 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 6171107 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 9549157 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  6.102592468261719
3 번째 트리 높이:  9

3 - 1 번째 탐색, 찾는 숫자 709764 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 6981443 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 1782043 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 4283788 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 1563011 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  5.450373888015747
4 번째 트리 높이:  9

4 - 1 번째 탐색, 찾는 숫자 3839687 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 9540648 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 446310 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 7984811 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자 477